# RAG 基礎入門

本範例展示：
1. **第1個儲存格**：建立向量資料庫
2. **第2個儲存格**：查詢向量資料庫

學習目標：理解如何將文本轉換為向量並進行相似度檢索

In [ ]:
# 第1個儲存格：建立向量資料庫

import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義包含文字檔案的目錄和持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir, "books", "三國演義.txt")
persistent_directory = os.path.join(current_dir, "db", "chroma_db_chinese_nb")

# 檢查 Chroma 向量存儲是否已存在
if not os.path.exists(persistent_directory):
    print("持久化目錄不存在。正在初始化向量存儲...")

    # 確保文字檔案存在
    if not os.path.exists(file_path):
        raise FileNotFoundError(
            f"檔案 {file_path} 不存在。請檢查路徑。"
        )

    # 從檔案讀取文字內容
    loader = TextLoader(file_path)
    documents = loader.load()

    # 將文件分割成塊
    # chunk_size=1000: 每個文本區塊最多 1000 個字元
    # chunk_overlap=0: 區塊之間不重疊
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    # 顯示分割文件的資訊
    print("\n--- 文件塊資訊 ---")
    print(f"文件塊數量: {len(docs)}")
    print(f"範例塊:\n{docs[0].page_content}\n")

    # 建立嵌入模型
    print("\n--- 正在建立嵌入 ---")
    print("使用 Jina Embeddings v2（繁體中文開源模型）")
    embeddings = HuggingFaceEmbeddings(
        model_name="jinaai/jina-embeddings-v2-base-zh"
    )
    print("\n--- 完成建立嵌入 ---")

    # 建立向量存儲並自動持久化
    print("\n--- 正在建立向量存儲 ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory)
    print("\n--- 完成建立向量存儲 ---")

else:
    print("向量存儲已存在。無需初始化。")

In [ ]:
# 第2個儲存格：查詢向量資料庫

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
persistent_directory = os.path.join(current_dir, "db", "chroma_db_chinese_nb")

# 定義嵌入模型（使用開源繁體中文模型）
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 使用嵌入函數載入現有的向量存儲
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

# 定義使用者的問題
query = "劉備是誰？"

# 根據查詢檢索相關文件
# search_type="similarity_score_threshold": 使用相似度分數閾值過濾
# k=3: 返回最多 3 個相關文件
# score_threshold=0.9: 只返回相似度分數 >= 0.9 的文件
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.9},
)
relevant_docs = retriever.invoke(query)

# 顯示相關結果及元數據
print("\n--- 相關文件 ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"文件 {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"來源: {doc.metadata.get('source', 'Unknown')}\n")